In [2]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB

from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, confusion_matrix)

In [4]:
# 1) Load the dataset
# --------------------------------
df = pd.read_csv("mail_l7_dataset.csv")

In [5]:
# Handle missing values
df = df.where(pd.notnull(df), "")

In [6]:
# Encode labels: spam=0, ham=1
df.loc[df["Category"].str.lower().str.strip() == "spam", "Category"] = 0
df.loc[df["Category"].str.lower().str.strip() == "ham",  "Category"] = 1
df["Category"] = df["Category"].astype(int)

print("=== SAMPLE DATA ===")
print(df.head())


=== SAMPLE DATA ===
   Category                                            Message
0         1  Go until jurong point, crazy.. Available only ...
1         1                      Ok lar... Joking wif u oni...
2         0  Free entry in 2 a wkly comp to win FA Cup fina...
3         1  U dun say so early hor... U c already then say...
4         1  Nah I don't think he goes to usf, he lives aro...


In [13]:
# 2) Split features (X) and target (y)
# --------------------------------
X = df["Message"].astype(str)
y = df["Category"].astype(int)

RANDOM_STATE = 42
# Train/test split (80/20, stratified)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=RANDOM_STATE, stratify=y
)

print("\n=== SPLIT SIZES ===")
print("Train:", X_train.shape[0], " | Test:", X_test.shape[0])



=== SPLIT SIZES ===
Train: 4457  | Test: 1115


In [12]:
# 3) TF-IDF Features
# --------------------------------
tfidf = TfidfVectorizer(min_df=1, stop_words="english", lowercase=True)
X_train_features = tfidf.fit_transform(X_train)
X_test_features  = tfidf.transform(X_test)

print("\n=== TF-IDF SHAPES ===")
print("X_train:", X_train_features.shape, " | X_test:", X_test_features.shape)


=== TF-IDF SHAPES ===
X_train: (4457, 7473)  | X_test: (1115, 7473)


In [14]:
# 4) Train Models
# --------------------------------
# Logistic Regression
lr = LogisticRegression(max_iter=1000, random_state=RANDOM_STATE)
lr.fit(X_train_features, y_train)
lr_pred = lr.predict(X_test_features)

In [15]:
# Random Forest
rf = RandomForestClassifier(n_estimators=200, random_state=RANDOM_STATE)
rf.fit(X_train_features, y_train)
rf_pred = rf.predict(X_test_features.toarray())

In [16]:

# Naive Bayes
nb = MultinomialNB()
nb.fit(X_train_features, y_train)
nb_pred = nb.predict(X_test_features)


In [17]:
# 5) Helper Functions
# --------------------------------
def print_clf_metrics(name, y_true, y_pred, pos_label=0):
    """Print Accuracy, Precision, Recall, F1. pos_label=0 means spam is positive."""
    acc  = accuracy_score(y_true, y_pred)
    prec = precision_score(y_true, y_pred, pos_label=pos_label)
    rec  = recall_score(y_true, y_pred, pos_label=pos_label)
    f1   = f1_score(y_true, y_pred, pos_label=pos_label)
    print(f"\n{name} Performance:")
    print(f"  Accuracy : {acc:.3f}")
    print(f"  Precision: {prec:.3f}  (positive = spam=0)")
    print(f"  Recall   : {rec:.3f}  (positive = spam=0)")
    print(f"  F1-Score : {f1:.3f}  (positive = spam=0)")

def print_confmat(name, y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred, labels=[1, 0])
    cm_df = pd.DataFrame(
        cm,
        index   = ["Actual: Ham (1)",  "Actual: Spam (0)"],
        columns = ["Pred: Ham (1)",    "Pred: Spam (0)"]
    )
    print(f"\n{name} – Confusion Matrix:\n{cm_df}")

In [18]:
# 6) Results for all models
# --------------------------------
print_clf_metrics("Logistic Regression", y_test, lr_pred, pos_label=0)
print_confmat("Logistic Regression", y_test, lr_pred)

print_clf_metrics("Random Forest", y_test, rf_pred, pos_label=0)
print_confmat("Random Forest", y_test, rf_pred)

print_clf_metrics("Naive Bayes", y_test, nb_pred, pos_label=0)
print_confmat("Naive Bayes", y_test, nb_pred)



Logistic Regression Performance:
  Accuracy : 0.960
  Precision: 0.991  (positive = spam=0)
  Recall   : 0.705  (positive = spam=0)
  F1-Score : 0.824  (positive = spam=0)

Logistic Regression – Confusion Matrix:
                  Pred: Ham (1)  Pred: Spam (0)
Actual: Ham (1)             965               1
Actual: Spam (0)             44             105

Random Forest Performance:
  Accuracy : 0.975
  Precision: 0.992  (positive = spam=0)
  Recall   : 0.819  (positive = spam=0)
  F1-Score : 0.897  (positive = spam=0)

Random Forest – Confusion Matrix:
                  Pred: Ham (1)  Pred: Spam (0)
Actual: Ham (1)             965               1
Actual: Spam (0)             27             122

Naive Bayes Performance:
  Accuracy : 0.965
  Precision: 1.000  (positive = spam=0)
  Recall   : 0.738  (positive = spam=0)
  F1-Score : 0.849  (positive = spam=0)

Naive Bayes – Confusion Matrix:
                  Pred: Ham (1)  Pred: Spam (0)
Actual: Ham (1)             966               0
Ac

In [20]:
# --------------------------------
# 9) Single-message sanity check (extended for 3 models)
# --------------------------------
i = 5  # change index to inspect different emails from X_test
sample_text = X_test.iloc[i]
true_label  = y_test.iloc[i]

# Predictions
lr_pred_one = int(lr.predict(tfidf.transform([sample_text]))[0])
rf_pred_one = int(rf.predict(tfidf.transform([sample_text]).toarray())[0])
nb_pred_one = int(nb.predict(tfidf.transform([sample_text]))[0])

def lab2str(v):
    return "Spam (0)" if v == 0 else "Ham (1)"

print("\n=== SINGLE MESSAGE CHECK ===")
snippet = (sample_text[:160] + "...") if len(sample_text) > 160 else sample_text
print("Text snippet:", snippet)
print("Actual      :", lab2str(true_label))
print("LR Pred     :", lab2str(lr_pred_one))
print("RF Pred     :", lab2str(rf_pred_one))
print("NB Pred     :", lab2str(nb_pred_one))



=== SINGLE MESSAGE CHECK ===
Text snippet: U have a secret admirer who is looking 2 make contact with U-find out who they R*reveal who thinks UR so special-call on 09058094565
Actual      : Spam (0)
LR Pred     : Spam (0)
RF Pred     : Spam (0)
NB Pred     : Spam (0)
